In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your openai api key"

In [1]:
openai_api_key ="your openai api key"

In [ ]:
!pip install -U langchain
!pip install -U openai
!pip install -U ragas
!pip install chromadb
!pip install lark
!pip install lark-parser
!pip install tiktoken
!pip install -U langchain-openai
!pip install openai
!pip install ragas

In [100]:
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
)
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from tqdm import tqdm
import random
import pandas as pd
from datasets import Dataset
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.chains import RetrievalQA
from langchain.prompts.chat import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.llms import OpenAI
import openai

**use openai embedding**

In [98]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

**define LLM**

In [99]:
llm = ChatOpenAI(temperature=0,openai_api_key=openai_api_key)

**Movie data preparation and preprocessing**

In [71]:
movie= pd.read_csv('iranian_movies.csv', encoding='utf-8')

In [72]:
#Remove NA's
movie = movie.dropna()

In [101]:
#Combine movie's informations
movie['combined_info'] = movie.apply(lambda row: f"نام فیلم: {row['Name']}. موضوع فیلم: {row['About']} ژانر: {row['Genre']} بازیگران: {row['Actors']} امتیاز فیلم: {row['Score']} عوامل فیلم: {row['Crew']} تعداد نفراتی که به این فیلم رای اده اند: {row['Voted']}", axis=1)

In [74]:
#Save processed dataset - combined_info for Langchain
movie[['combined_info']].to_csv('movie_updated.csv', index=False)

In [75]:
pd.read_csv('/content/movie_updated.csv', encoding='utf-8')

,combined_info
0,نام فیلم: یکی از میان همه. موضوع فیلم: نیلوفر...
1,نام فیلم: جاده جنایت. موضوع فیلم: وقتی کتایون...
2,نام فیلم: ساوالان. موضوع فیلم: روستای ساوالان...
3,نام فیلم: پوزه چرمی. موضوع فیلم: مردی برای رس...
4,نام فیلم: شکلاتی. موضوع فیلم: فیلم شکلاتی درب...
...,...
920,نام فیلم: جنگلبان. موضوع فیلم: در اوائل انقلا...
921,نام فیلم: بازنشسته ها. موضوع فیلم: قصه مردی ا...
922,نام فیلم: جدا افتاده. موضوع فیلم: دختری روستا...
923,نام فیلم: پنجمین سوار سرنوشت. موضوع فیلم: محس...


No charts were generated by quickchart


# **Retriever**

In [102]:
#data loader
loader = CSVLoader(file_path="iranian_movies.csv",encoding='utf-8')
data = loader.load()

#data transformers
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

#embeddings model, this can be a local LLM as well
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
llm = OpenAI(openai_api_key=openai_api_key)

#Vector Database
docsearch = Chroma.from_documents(texts, embeddings)

**test retriever**

In [78]:
query = "من دنبال یه هستم که در مورد روستای ساوالان باشه و اکشن باشه. میشه بهم پیشنهاد بدی؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="combined_info: Title:  ساوالان. Overview: روستای ساوالان هرسال توسط گروهی راهزن غارت می شود. اهالی روستا گروهی را برای کمک می طلبند جوانی به نام سلام به این گروه ملحق می شود. ولی گروهی که برای کمک آمده اند باعث اتفاقات دیگری می شوند. Genres: ['اکشن', 'ماجراجویی']", metadata={'row': 2, 'source': '/content/movie_updated.csv'})]

In [79]:
query = "ژانر فیلم بچه تابستان چیست؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: بچه تابستان\nActors: [' مهدی شیخوند', ' آرمان مرادی']\nScore: 0.66\nAbout: این نمایش، کاری از گروه تئاتر جیرجیرک به کارگردانی دریا نظری می باشد.در خلاصه داستان این نمایش آمده است : من توی پارکم، توی دستم هیچی نیست؛کی یک چاقو تو دست من گذاشت؟\nGenre: ['فیلم تئاتر', 'خانوادگی']\nCrew: [' دریا نظری', ' حسن حامد']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_17045_1-m.jpg\nVoted: 149\nType: فیلم", metadata={'row': 35, 'source': 'iranian_movies.csv'})]

In [80]:
query = "ژانر فیلم دزدان خیابان جردن چیست؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: دزدان خیابان جردن\nActors: [' فتحعلی اویسی', ' ارژنگ امیرفضلی', ' رضا شفیعی جم', ' حدیث فولادوند', ' رابعه اسکویی']\nScore: 0.77\nAbout: فیلم دزدان خبابان جردن داستان سامی سه سوت و هوشنگ است که قصد سرقت از یک گاو صندوق را در خیابان جردن دارند و ماجراهای عجیب و غریبی برایشان پیش می آید و…\nGenre: ['کمدی', 'خانوادگی']\nCrew: [' وحید اسلامی', ' داریوش بابائیان', ' محمد مجیدی']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_11989_20028-m.jpg\nVoted: 1948\nType: فیلم", metadata={'row': 1106, 'source': 'iranian_movies.csv'})]

In [89]:
query = "امتیاز فیلم گاهی واقعی چند است؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: گاهی واقعی\nActors: [' تیم رزان', ' میترا لهراسب', ' آهو خردمند', ' حدیث فولادوند', ' منیژه محامدی', ' مهتاج نجومی', ' ثریا حلی']\nScore: 0.74\nAbout: فیلم داستان انسانی را روایت می كند كه برای یافتن هویت اصلی اش به كشورش برمی گردد.\nGenre: ['خانوادگی', 'اجتماعی']\nCrew: [' رامین لباسچی', ' رامین لباسچی', ' داریوش بابائیان', ' فریبرز لاچینی', ' فرهاد صبا']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_2517_895-m.jpg\nVoted: 168\nType: فیلم", metadata={'row': 31, 'source': 'iranian_movies.csv'})]

In [90]:
query = "چه کسانی در فیلم شکلاتی بازی کرده اند؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content='فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد.', metadata={'Actors': ' شبنم مقدمی و  ناصر هاشمی و  محمدرضا هدایتی و  ارژنگ امیرفضلی و  فرزین محدث و  علیرضا مسعودی', 'Crew': ' سهیل موفق و  کریم خودسیانی و  ایرج محمدی', 'Genre': 'خانوادگی و کمدی', 'Name': ' شکلاتی', 'Score': 0.9, 'Type': 'فیلم', 'Voted': 18330})]

In [91]:
query = "موضوع فیلم ترانه کوچک زندگی چیست؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: ترانه کوچک زندگی\nActors: [' عنایت بخشی', ' مهدی امینی خواه', ' زهرا اویسی', ' لعیا میر نصیری', ' بهرام عربیان']\nScore: 0.75\nAbout: فیلم ترانه کوچک زندگی ، با محوریت مواد مخدر و متلاشی شدن بنیان خانواده در اثر اعتیاد می باشد...\nGenre: ['خانوادگی', 'اجتماعی']\nCrew: [' قادر قنبرزاده', ' سید محمد قاضی', ' محمود زینال زاده', ' امیر سلیمانی فاخر', ' افشین آذری']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_19932_475-m.jpg\nVoted: 199\nType: فیلم", metadata={'row': 39, 'source': 'iranian_movies.csv'})]

In [104]:
query = "تعداد نفراتی که به فیلم گاهی واقعی رای داده اند؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: گاهی واقعی\nActors: [' تیم رزان', ' میترا لهراسب', ' آهو خردمند', ' حدیث فولادوند', ' منیژه محامدی', ' مهتاج نجومی', ' ثریا حلی']\nScore: 0.74\nAbout: فیلم داستان انسانی را روایت می كند كه برای یافتن هویت اصلی اش به كشورش برمی گردد.\nGenre: ['خانوادگی', 'اجتماعی']\nCrew: [' رامین لباسچی', ' رامین لباسچی', ' داریوش بابائیان', ' فریبرز لاچینی', ' فرهاد صبا']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_2517_895-m.jpg\nVoted: 168\nType: فیلم", metadata={'row': 31, 'source': 'iranian_movies.csv'})]

In [105]:
query = "موضوع فیلم تمرین برای اجرا چیه؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: تمرین برای اجرا\nActors: [' میکائیل شهرستانی', ' مهدی میامی', ' محمدعلی سجادی', ' محمود راسخ فر', ' محمد ساربان', ' حسین فلاح']\nScore: 0.59\nAbout: یک کارگردان تئاتر تصمیم می گیرد تا با جمع کردن گروهی بتواند داستان سیاوش را از روی شاهنامه فردوسی بخوانند و برای اجرا تمرین کنند…\nGenre: ['تاریخی | مذهبی']\nCrew: [' محمدرضا سکوت', ' محمدعلی سجادی', ' محمدعلی سجادی', ' محمدعلی سجادی']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_26069_1-m.jpg\nVoted: 111\nType: فیلم", metadata={'row': 15, 'source': 'iranian_movies.csv'})]

In [107]:
query = "آیا نازنین کریمی در فیلم فریاد در شب بازی کرده؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="Name: فریاد در شب\nActors: [' ایرج نوذری', ' حدیث فولادوند', ' نازنین کریمی', ' محمود مقامی', ' زهره حمیدی']\nScore: 0.69\nAbout: دختری به نام هما که در فراز و نشیب زندگی اثیر حوادثی می شود که زندگی او را از سیر طبیعی خود خارج کرده و او را دچار کابوس می کند. همزمان او در گیر و دار قتل هایی قرار می گیرد که پلیس به دنبال پیداکردن سرنخی برای سرمنشأ این قتل هاست.\nGenre: ['خانوادگی', 'اجتماعی']\nCrew: [' کریم رجبی', ' هایده رزم آور', ' مسعود عسگری']\nImage: https://static.cdn.asset.filimo.com/flmt/mov_11376_14525-m.jpg\nVoted: 224\nType: فیلم", metadata={'row': 8, 'source': 'iranian_movies.csv'})]

In [108]:
query = "یه فیلم که ژانر اکشن داشته باشه"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="combined_info: Title:  شب زده. Overview: توجه راننده کامیونی در راه مانده بر اثر داد و فریاد یک نفر به زیر یک پل جلب می شود. جایی که یک نفر مردی زانو زده را می کشد .یک سال بعد … Genres: ['اکشن', 'پلیسی معمایی']", metadata={'row': 64, 'source': '/content/movie_updated.csv'})]

In [116]:
query = "موضوع فیلم اسرافیل چیه؟"
docs = docsearch.similarity_search(query, k=1)
docs

[Document(page_content="combined_info: Title:  اسرافیل. Overview: فیلم اسرافیل ، درباره چالش ها و نیازهای عاطفی و روابط انسانی است و در خلاصه داستان این فیلم آمده است؛مردی که ماهی را دوست داشت بازگشته است. دختری که بهروز را دوست دارد باز می گردد...مردگان زنده می شوند. Genres: ['خانوادگی', 'اجتماعی']", metadata={'row': 853, 'source': '/content/movie_updated.csv'})]

# **QA Chain**

In [95]:
template = """You are an assistant for Movie question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise. you have to translate your answer in Persian

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=openai_api_key)

retrieval_augmented_qa_chain = RetrievalQA.from_chain_type(llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

**Test retrieval augmented QA chain**

In [97]:
result = retrieval_augmented_qa_chain.invoke(query)
result['result']

'موضوع فیلم ترانه کوچک زندگی متمرکز بر مواد مخدر و متلاشی شدن بنیان خانواده در اثر اعتیاد است.'

# **Create testset for evaluation**

In [187]:
test_sentences = [
    Document(
        page_content="فیلم ساوالان بازیگران آن مجید مظفری و  محمد برسوزیان و  فردوس کاویانی و  خسرو دستگیر هستند و ژانر این فیلم اکشن و ماجراجویی است و امتیاز این فیلم 0.9 است. 493 نفر به این فیلم رای داده اند. موضوع این فیلم روستای ساوالان هرسال توسط گروهی راهزن غارت می شود. اهالی روستا گروهی را برای کمک می طلبند جوانی به نام سلام به این گروه ملحق می شود. ولی گروهی که برای کمک آمده اند باعث اتفاقات دیگری می شوند. است. از عوامل این فیلم میتوان به یدالله صمدی و  سیفعلی سیدکوثری نژاد و  یدالله صمدی و  فرهاد فخرالدینی و  حسن قلی زاده اشاره کرد."
    ),
    Document(
        page_content="فیلم بچه تابستان بازیگران آن مهدی شیخوند و  آرمان مرادی هستند و ژانر این فیلم فیلم تئاتر و خانوادگی است و امتیاز این فیلم 0.66 است. 149 نفر به این فیلم رای داده اند. موضوع این فیلم این نمایش، کاری از گروه تئاتر جیرجیرک به کارگردانی دریا نظری می باشد.در خلاصه داستان این نمایش آمده است : من توی پارکم، توی دستم هیچی نیست؛کی یک چاقو تو دست من گذاشت؟ است. از عوامل این فیلم میتوان به دریا نظری و  حسن حامد اشاره کرد."
    ),
    Document(
        page_content="فیلم دزدان خیابان جردن بازیگران آن فتحعلی اویسی و  ارژنگ امیرفضلی و  رضا شفیعی جم و  حدیث فولادوند و  رابعه اسکویی هستند و ژانر این فیلم کمدی و خانوادگی است و امتیاز این فیلم 0.77 است. 1948 نفر به این فیلم رای داده اند. موضوع این فیلم فیلم دزدان خبابان جردن داستان سامی سه سوت و هوشنگ است که قصد سرقت از یک گاو صندوق را در خیابان جردن دارند و ماجراهای عجیب و غریبی برایشان پیش می آید و… است. از عوامل این فیلم میتوان به وحید اسلامی و  داریوش بابائیان و  محمد مجیدی اشاره کرد."
    ),
    Document(
        page_content="فیلم گاهی واقعی بازیگران آن تیم رزان و  میترا لهراسب و  آهو خردمند و  حدیث فولادوند و  منیژه محامدی و  مهتاج نجومی و  ثریا حلی هستند و ژانر این فیلم خانوادگی و اجتماعی است و امتیاز این فیلم 0.74 است. 168 نفر به این فیلم رای داده اند. موضوع این فیلم فیلم داستان انسانی را روایت می كند كه برای یافتن هویت اصلی اش به كشورش برمی گردد. است. از عوامل این فیلم میتوان به رامین لباسچی و  رامین لباسچی و  داریوش بابائیان و  فریبرز لاچینی و  فرهاد صبا اشاره کرد."
    ),
    Document(
        page_content="فیلم تمرین برای اجرا بازیگران آن میکائیل شهرستانی و  مهدی میامی و  محمدعلی سجادی و  محمود راسخ فر و  محمد ساربان و  حسین فلاح هستند و ژانر این فیلم تاریخی | مذهبی است و امتیاز این فیلم 0.59 است. 111 نفر به این فیلم رای داده اند. موضوع این فیلم یک کارگردان تئاتر تصمیم می گیرد تا با جمع کردن گروهی بتواند داستان سیاوش را از روی شاهنامه فردوسی بخوانند و برای اجرا تمرین کنند… است. از عوامل این فیلم میتوان به محمدرضا سکوت و  محمدعلی سجادی و  محمدعلی سجادی و  محمدعلی سجادی اشاره کرد."
    ),
    Document(
        page_content="فیلم فریاد در شب بازیگران آن ایرج نوذری و  حدیث فولادوند و  نازنین کریمی و  محمود مقامی و  زهره حمیدی هستند و ژانر این فیلم خانوادگی و اجتماعی است و امتیاز این فیلم 0.69 است. 224 نفر به این فیلم رای داده اند. موضوع این فیلم دختری به نام هما که در فراز و نشیب زندگی اثیر حوادثی می شود که زندگی او را از سیر طبیعی خود خارج کرده و او را دچار کابوس می کند. همزمان او در گیر و دار قتل هایی قرار می گیرد که پلیس به دنبال پیداکردن سرنخی برای سرمنشأ این قتل هاست. است. از عوامل این فیلم میتوان به کریم رجبی و  هایده رزم آور و  مسعود عسگری اشاره کرد."
    ),
    Document(
        page_content="فیلم شب زده بازیگران آن بابک حمیدیان و  طناز طباطبایی و  رحیم نوروزی و  آرش تاج تهرانی و  کیانوش گرامی و  بهرام ابراهیمی و  سیروس میمنت و  فرهاد بشارتی و  افسانه ناصری هستند و ژانر این فیلم اکشن و پلیسی معمایی است و امتیاز این فیلم 0.73 است. 443 نفر به این فیلم رای داده اند. موضوع این فیلم توجه راننده کامیونی در راه مانده بر اثر داد و فریاد یک نفر به زیر یک پل جلب می شود. جایی که یک نفر مردی زانو زده را می کشد .یک سال بعد … است. از عوامل این فیلم میتوان به فرزاد موتمن و  سعید شاهسواری و  مهدی آزادی و  مسعود صباح و  کیوان جهانشاهی اشاره کرد."
    ),
    Document(
        page_content="فیلم ترانه کوچک زندگی بازیگران آن عنایت بخشی و  مهدی امینی خواه و  زهرا اویسی و  لعیا میر نصیری و  بهرام عربیان هستند و ژانر این فیلم خانوادگی و اجتماعی است و امتیاز این فیلم 0.75 است. 199 نفر به این فیلم رای داده اند. موضوع این فیلم فیلم ترانه کوچک زندگی ، با محوریت مواد مخدر و متلاشی شدن بنیان خانواده در اثر اعتیاد می باشد... است. از عوامل این فیلم میتوان به قادر قنبرزاده و  سید محمد قاضی و  محمود زینال زاده و  امیر سلیمانی فاخر و  افشین آذری اشاره کرد."
    ),
    Document(
        page_content="فیلم اسرافیل بازیگران آن هدیه تهرانی و  پژمان بازغی و  هدی زین العابدین و  مریلا زارعی و  علی عمرانی و  پوریا رحیمی سام و  سهیلا رضوی هستند و ژانر این فیلم خانوادگی و اجتماعی است و امتیاز این فیلم 0.86 است. 10497 نفر به این فیلم رای داده اند. موضوع این فیلم فیلم اسرافیل ، درباره چالش ها و نیازهای عاطفی و روابط انسانی است و در خلاصه داستان این فیلم آمده است؛مردی که ماهی را دوست داشت بازگشته است. دختری که بهروز را دوست دارد باز می گردد...مردگان زنده می شوند. است. از عوامل این فیلم میتوان به آیدا پناهنده و  مستانه مهاجر و  مرتضی قیدی و  ارسلان امیری و  آیدا پناهنده و  مجید پوستی اشاره کرد."
    ),
    Document(
        page_content="فیلم شکلاتی بازیگران آن شبنم مقدمی و  ناصر هاشمی و  محمدرضا هدایتی و  ارژنگ امیرفضلی و  فرزین محدث و  علیرضا مسعودی هستند و ژانر این فیلم خانوادگی و کمدی است و امتیاز این فیلم 0.9 است. 18330 نفر به این فیلم رای داده اند. موضوع این فیلم فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد. است. از عوامل این فیلم میتوان به سهیل موفق و  کریم خودسیانی و  ایرج محمدی اشاره کرد."
    )
]

# **Generating test questions using GPT**

In [188]:
question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [189]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)

format_instructions = question_output_parser.get_format_instructions()

In [190]:
question_generation_llm = ChatOpenAI(model="gpt-3.5-turbo-1106", openai_api_key=openai_api_key)

bare_prompt_template = "{content}"

bare_template = ChatPromptTemplate.from_template(template=bare_prompt_template)

In [191]:
qa_template = """\
You are interested in the field of movies and series. For each context, create a simple question about the movie, for example, about the topic of the movie - the genre of the movie - who starred in that movie - how many people have voted - what is the score of that movieand translate it to persian language that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=test_sentences[0],
    format_instructions=format_instructions
)

question_generation_chain = bare_template | question_generation_llm

response = question_generation_chain.invoke({"content" : messages})

output_dict = question_output_parser.parse(response.content)

# **Generating context**

In [156]:
random.seed(42)

qac_triples = []

# randomly select
for text in tqdm(random.sample(test_sentences, 10)):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = question_generation_chain.invoke({"content" : messages})
  try:
    output_dict = question_output_parser.parse(response.content)
  except Exception as e:
    continue
  output_dict["context"] = text
  qac_triples.append(output_dict)

100%|██████████| 10/10 [01:17<00:00,  7.77s/it]


In [157]:
qac_triples[2]

{'question': 'فیلم تمرین برای اجرا بازیگران آن چه کسانی هستند؟',
 'context': Document(page_content='فیلم تمرین برای اجرا بازیگران آن میکائیل شهرستانی و  مهدی میامی و  محمدعلی سجادی و  محمود راسخ فر و  محمد ساربان و  حسین فلاح هستند و ژانر این فیلم تاریخی | مذهبی است و امتیاز این فیلم 0.59 است. 111 نفر به این فیلم رای داده اند. موضوع این فیلم یک کارگردان تئاتر تصمیم می گیرد تا با جمع کردن گروهی بتواند داستان سیاوش را از روی شاهنامه فردوسی بخوانند و برای اجرا تمرین کنند… است. از عوامل این فیلم میتوان به محمدرضا سکوت و  محمدعلی سجادی و  محمدعلی سجادی و  محمدعلی سجادی اشاره کرد.')}

# **Generate “ground truth” answers using GPT-4**

In [158]:
answer_generation_llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0,openai_api_key=openai_api_key)

answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)

format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

answer_generation_chain = bare_template | answer_generation_llm

response = answer_generation_chain.invoke({"content" : messages})

output_dict = answer_output_parser.parse(response.content)

In [161]:
for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = answer_generation_chain.invoke({"content" : messages})
  try:
    output_dict = answer_output_parser.parse(response.content)
  except Exception as e:
    continue
  triple["answer"] = output_dict["answer"]

100%|██████████| 10/10 [01:59<00:00, 11.97s/it]


# **Combine questions, contexts, and answers**

In [162]:
ground_truth_qac_set = pd.DataFrame(qac_triples)

ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))

ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})

eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [163]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 10
})

In [165]:
eval_dataset[1]

{'question': 'فیلم ساوالان چه ژانری است؟',
 'context': 'فیلم ساوالان بازیگران آن مجید مظفری و  محمد برسوزیان و  فردوس کاویانی و  خسرو دستگیر هستند و ژانر این فیلم اکشن و ماجراجویی است و امتیاز این فیلم 0.9 است. 493 نفر به این فیلم رای داده اند. موضوع این فیلم روستای ساوالان هرسال توسط گروهی راهزن غارت می شود. اهالی روستا گروهی را برای کمک می طلبند جوانی به نام سلام به این گروه ملحق می شود. ولی گروهی که برای کمک آمده اند باعث اتفاقات دیگری می شوند. است. از عوامل این فیلم میتوان به یدالله صمدی و  سیفعلی سیدکوثری نژاد و  یدالله صمدی و  فرهاد فخرالدینی و  حسن قلی زاده اشاره کرد.',
 'ground_truth': 'ژانر فیلم ساوالان اکشن و ماجراجویی است.'}

In [166]:
eval_dataset[2]

{'question': 'فیلم تمرین برای اجرا بازیگران آن چه کسانی هستند؟',
 'context': 'فیلم تمرین برای اجرا بازیگران آن میکائیل شهرستانی و  مهدی میامی و  محمدعلی سجادی و  محمود راسخ فر و  محمد ساربان و  حسین فلاح هستند و ژانر این فیلم تاریخی | مذهبی است و امتیاز این فیلم 0.59 است. 111 نفر به این فیلم رای داده اند. موضوع این فیلم یک کارگردان تئاتر تصمیم می گیرد تا با جمع کردن گروهی بتواند داستان سیاوش را از روی شاهنامه فردوسی بخوانند و برای اجرا تمرین کنند… است. از عوامل این فیلم میتوان به محمدرضا سکوت و  محمدعلی سجادی و  محمدعلی سجادی و  محمدعلی سجادی اشاره کرد.',
 'ground_truth': 'بازیگران فیلم تمرین برای اجرا عبارتند از: میکائیل شهرستانی، مهدی میامی، محمدعلی سجادی، محمود راسخ فر، محمد ساربان، و حسین فلاح.'}

# **create our dataset for RAGAS**

In [167]:
# create our dataset for RAGAS
questions = []
answers = []

# Iterate through each row in the dataset
for row in eval_dataset:
    questions.append(row['question'])

# Iterate through each question and get answers
for question in eval_dataset["question"]:
    result = retrieval_augmented_qa_chain.invoke(question)
    answer = result['result']
    answers.append(answer)

In [168]:
# ground_truths should be in type Sequence[string]

ground_truths = []
for row in eval_dataset:
    formatted_ground_truth = [row['ground_truth']]  # Wrap the string in a list
    ground_truths.append(formatted_ground_truth)

# Output the result
print(ground_truths)

[['149 نفر به این فیلم رای داده اند.'], ['ژانر فیلم ساوالان اکشن و ماجراجویی است.'], ['بازیگران فیلم تمرین برای اجرا عبارتند از: میکائیل شهرستانی، مهدی میامی، محمدعلی سجادی، محمود راسخ فر، محمد ساربان، و حسین فلاح.'], ['ژانر فیلم شکلاتی خانوادگی و کمدی است.'], ['امتیاز فیلم شب زده 0.73 است.'], ['ژانر فیلم فریاد در شب خانوادگی و اجتماعی است.'], ['10497 نفر به فیلم اسرافیل رای داده اند.'], ['کمدی و خانوادگی'], ['ژانر فیلم گاهی واقعی خانوادگی و اجتماعی است.'], ['199 نفر به این فیلم رای داده اند.']]


In [169]:
# retrieved context should be in type Sequence[string]

retrieved_documents = []

# Define the Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

def format_document(doc):
    page_content = doc.page_content
    metadata_str = ', '.join([f"{key}: '{value}'" for key, value in doc.metadata.items()])
    return f"'{page_content}', metadata={{{metadata_str}}}"

for query in questions:
    documents = docsearch.similarity_search(query, k=1)  # Retrieve documents for each query
    formatted_docs = [format_document(doc) for doc in documents]  # Format all documents for a query
    retrieved_documents.append(formatted_docs)  # Append the list of formatted documents

# Output the result
print(retrieved_documents)


[["'combined_info: Title:  سوم شخص غایب. Overview: فیلم سوم شخص غایب در ژانر کودک محصول سال 1397 به کارگردانی وحید نیکخواه آزاد است. در خلاصه داستان این فیلم آمده است که رشید در امتحانات شهریور مردود شده و پدرش اجازه ادامه تحصیل نمی دهد. دوستان او، امیر و سعید، خود را به آب و آتش می زنند تا مانع از ترک تحصیل دوست شان بشوند. Genres: ['خانوادگی', 'کودک']', metadata={row: '624', source: '/content/movie_updated.csv'}"], ["'فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد.', metadata={Actors: ' شبنم مقدمی و  ناصر هاشمی و  محمدرضا هدایتی و  ارژنگ امیرفضلی و  فرزین محدث و  علیرضا مسعودی', Crew: ' سهیل موفق و  کریم خودسیانی و  ایرج محمدی', Genre: 'خانوادگی و کمدی', Name: ' شکلاتی', Score: '0.9', Type: 'فیلم', Voted: '18330'}"], ["'فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد.', metadata={Actors: ' شبنم مقدمی و  ناصر 

In [170]:
# Creating the new dictionary format
data = {
    "question": questions,
    "contexts": retrieved_documents,
    "ground_truths": ground_truths,
    "answer": answers
}
# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [171]:
dataset

Dataset({
    features: ['question', 'contexts', 'ground_truths', 'answer'],
    num_rows: 10
})

In [172]:
dataset["question"]

['چند نفر به این فیلم رای داده اند؟',
 'فیلم ساوالان چه ژانری است؟',
 'فیلم تمرین برای اجرا بازیگران آن چه کسانی هستند؟',
 'فیلم شکلاتی چه ژانری است؟',
 'چه جوری امتیاز فیلم شب زده است؟',
 'فیلم فریاد در شب چه ژانری است؟',
 'چند نفر به فیلم اسرافیل رای داده اند؟',
 'فیلم دزدان خیابان جردن چه ژانری است؟',
 'فیلم گاهی واقعی چه ژانری است؟',
 'چند نفر به این فیلم رای داده اند؟']

In [173]:
dataset["answer"]

['2465 نفر به این فیلم رای داده اند.',
 'فیلم شکلاتی یک فیلم کمدی است.',
 'من نمی\u200cدانم.',
 'فیلم شکلاتی یک فیلم کمدی است.',
 'امتیاز فیلم شب زده 0.84 است.',
 'من نمی\u200cدانم.',
 'تعداد ۱۰۴۹۷ نفر به فیلم اسرافیل رای داده اند.',
 'ژانر فیلم دزدان خیابان جردن کمدی و خانوادگی است.',
 'فیلم گاهی واقعی ژانر خانوادگی و اجتماعی دارد.',
 '2465 نفر به این فیلم رای داده اند.']

In [174]:
dataset["contexts"]

[["'combined_info: Title:  سوم شخص غایب. Overview: فیلم سوم شخص غایب در ژانر کودک محصول سال 1397 به کارگردانی وحید نیکخواه آزاد است. در خلاصه داستان این فیلم آمده است که رشید در امتحانات شهریور مردود شده و پدرش اجازه ادامه تحصیل نمی دهد. دوستان او، امیر و سعید، خود را به آب و آتش می زنند تا مانع از ترک تحصیل دوست شان بشوند. Genres: ['خانوادگی', 'کودک']', metadata={row: '624', source: '/content/movie_updated.csv'}"],
 ["'فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد.', metadata={Actors: ' شبنم مقدمی و  ناصر هاشمی و  محمدرضا هدایتی و  ارژنگ امیرفضلی و  فرزین محدث و  علیرضا مسعودی', Crew: ' سهیل موفق و  کریم خودسیانی و  ایرج محمدی', Genre: 'خانوادگی و کمدی', Name: ' شکلاتی', Score: '0.9', Type: 'فیلم', Voted: '18330'}"],
 ["'فیلم شکلاتی درباره این فیلم درباره کارخانه شکلات سازی است که به پارسای ۱۳ ساله ارث می رسد و پارسا با کمک دوستان خود سعی در اداره آن دارد.', metadata={Actors: ' شبنم مقدمی و  ناص

In [175]:
dataset.to_pandas()

,question,contexts,ground_truths,answer
0,چند نفر به این فیلم رای داده اند؟,['combined_info: Title: سوم شخص غایب. Overvie...,[149 نفر به این فیلم رای داده اند.],2465 نفر به این فیلم رای داده اند.
1,فیلم ساوالان چه ژانری است؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[ژانر فیلم ساوالان اکشن و ماجراجویی است.],فیلم شکلاتی یک فیلم کمدی است.
2,فیلم تمرین برای اجرا بازیگران آن چه کسانی هستند؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[بازیگران فیلم تمرین برای اجرا عبارتند از: میک...,من نمی‌دانم.
3,فیلم شکلاتی چه ژانری است؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[ژانر فیلم شکلاتی خانوادگی و کمدی است.],فیلم شکلاتی یک فیلم کمدی است.
4,چه جوری امتیاز فیلم شب زده است؟,['Name: رویای نیمه شب پاییز\nActors: [' امیر ج...,[امتیاز فیلم شب زده 0.73 است.],امتیاز فیلم شب زده 0.84 است.
5,فیلم فریاد در شب چه ژانری است؟,['combined_info: Title: جوانی. Overview: فیلم...,[ژانر فیلم فریاد در شب خانوادگی و اجتماعی است.],من نمی‌دانم.
6,چند نفر به فیلم اسرافیل رای داده اند؟,['combined_info: Title: اسرافیل. Overview: فی...,[10497 نفر به فیلم اسرافیل رای داده اند.],تعداد ۱۰۴۹۷ نفر به فیلم اسرافیل رای داده اند.
7,فیلم دزدان خیابان جردن چه ژانری است؟,['Name: دزدان خیابان جردن\nActors: [' فتحعلی ا...,[کمدی و خانوادگی],ژانر فیلم دزدان خیابان جردن کمدی و خانوادگی است.
8,فیلم گاهی واقعی چه ژانری است؟,"['Name: گاهی واقعی\nActors: [' تیم رزان', ' می...",[ژانر فیلم گاهی واقعی خانوادگی و اجتماعی است.],فیلم گاهی واقعی ژانر خانوادگی و اجتماعی دارد.
9,چند نفر به این فیلم رای داده اند؟,['combined_info: Title: سوم شخص غایب. Overvie...,[199 نفر به این فیلم رای داده اند.],2465 نفر به این فیلم رای داده اند.


# **RAGAS Evaluation Metrics**

In [193]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:03<00:00,  3.01s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:08<00:00,  8.02s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:12<00:00, 12.10s/it]


In [182]:
df.head()

,question,contexts,ground_truths,answer,context_precision,context_recall,faithfulness,answer_relevancy
0,چند نفر به این فیلم رای داده اند؟,['combined_info: Title: سوم شخص غایب. Overvie...,[149 نفر به این فیلم رای داده اند.],2465 نفر به این فیلم رای داده اند.,0.0,0.0,0.0,0.837088
1,فیلم ساوالان چه ژانری است؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[ژانر فیلم ساوالان اکشن و ماجراجویی است.],فیلم شکلاتی یک فیلم کمدی است.,1.0,0.0,1.0,0.875436
2,فیلم تمرین برای اجرا بازیگران آن چه کسانی هستند؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[بازیگران فیلم تمرین برای اجرا عبارتند از: میک...,من نمی‌دانم.,0.0,0.0,0.0,0.000000
3,فیلم شکلاتی چه ژانری است؟,['فیلم شکلاتی درباره این فیلم درباره کارخانه ش...,[ژانر فیلم شکلاتی خانوادگی و کمدی است.],فیلم شکلاتی یک فیلم کمدی است.,1.0,1.0,1.0,0.907777
4,چه جوری امتیاز فیلم شب زده است؟,['Name: رویای نیمه شب پاییز\nActors: [' امیر ج...,[امتیاز فیلم شب زده 0.73 است.],امتیاز فیلم شب زده 0.84 است.,0.0,1.0,1.0,0.891968


In [194]:
df.to_csv('ragas_result.csv')

In [201]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.1
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
asgiref==3.7.2
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.2.0
audioread==3.0.1
Authlib==1.3.0
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
backoff==2.2.1
bcrypt==4.1.2
beautifulsoup4==4.11.2
bidict==0.22.1
bigframes==0.19.2
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.42
branca==0.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
chroma-hnswlib==0.7.3
chromadb==0.4.22
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.0
colorcet==3.0.1
coloredlogs==15.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
co